# Get Candlestick Patterns

## Import relevant libraries

In [1]:
import os
import pandas as pd
import talib
import numpy as np

import datetime
from configparser import ConfigParser

## Set some 'global' parameters

In [2]:
configur = ConfigParser()
print (configur.read('../config_files/config_candlestick_patterns.ini'))
#print ("Sections : ", configur.sections())

['../config_files/config_candlestick_patterns.ini']


## Get and process the data

### Method that executes the process (run)

In [3]:
def run():
    try:
        
        # Parameters
        list_asset_ticket = ["BTCUSDT", "ETHUSDT", "BNBUSDT"]
        list_timestamp = ["1d", "1h", "30m", "15m", "5m"]
        start_date = "1 Jan, 2017"
        end_date = "31 Dec, 2022"

        # Candlesitck selected patterns
        candle_names = ['CDLINVERTEDHAMMER',
                        'CDLHAMMER',
                        'CDLPIERCING',
                        'CDLMORNINGSTAR',
                        'CDLSHOOTINGSTAR',
                        'CDLHANGINGMAN',
                        'CDLDARKCLOUDCOVER',
                        'CDLEVENINGSTAR',
                        'CDLENGULFING']

        for asset_ticket in list_asset_ticket:
            for timestamp in list_timestamp:
                
                # Data file parameters
                input_data_path = '../../data/05_raw_group'
                input_data_filename = "binance" + \
                                "_" + asset_ticket + \
                                "_" + timestamp + \
                                "_from_" + datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                                "_to_" + datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y_%m_%d')
                input_data_extension = ".csv"
                full_path_input_data = os.path.join(input_data_path, input_data_filename + input_data_extension)        

                df = pd.read_csv(full_path_input_data)

                # extract OHLC 
                op = df['open']
                hi = df['high']
                lo = df['low']
                cl = df['close']

                # create columns for each pattern
                for candle in candle_names:
                    # below is same as;
                    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
                    df[candle] = getattr(talib, candle)(op, hi, lo, cl)

                # Split CDLENGULFING into bullish and bearish
                list_open_time_bullish_engulfing = df[df['CDLENGULFING']==100]['date'].to_list()
                list_open_time_bearish_engulfing = df[df['CDLENGULFING']==-100]['date'].to_list()

                df['CDLENGULFINGBULLISH'] = np.where(df['date'].isin(list_open_time_bullish_engulfing), 100, 0)
                df['CDLENGULFINGBEARISH'] = np.where(df['date'].isin(list_open_time_bearish_engulfing), -100, 0)
                
                # Replace first (based on the 'window_size') Candlestick values for 0 
                window_size = configur.getint('candlestick_patterns', 'window_size')
                subset = ['CDLINVERTEDHAMMER', 'CDLHAMMER',
                            'CDLPIERCING', 'CDLMORNINGSTAR', 'CDLSHOOTINGSTAR', 'CDLHANGINGMAN',
                            'CDLDARKCLOUDCOVER', 'CDLEVENINGSTAR', 'CDLENGULFING',
                            'CDLENGULFINGBULLISH', 'CDLENGULFINGBEARISH']

                df.loc[:window_size, subset] = df.loc[:window_size, subset].replace([100, -100],[0, 0])

                # Export data parameters
                export_path = "../../data/10_candlesticks_signals_raw"
                export_filename = "binance" + \
                                "_" + asset_ticket + \
                                "_" + timestamp + \
                                "_from_" + datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                                "_to_" + datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                                "_candlesticks_signals_raw"
                export_extension = ".csv"
                full_export_path = os.path.join(export_path, export_filename + export_extension)

                # Export data
                df.to_csv(full_export_path, index=False)

    except Exception as e:
        print(e)

### Run!

In [4]:
run()